In [1]:
import torch
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader
from tqdm import tqdm  # For nice progress bar!

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        self.flat = nn.Flatten(-3, -1)
        self.fc1 = nn.Linear(43808, 4500)
        self.act2 = nn.ReLU()
        self.fc2 = nn.Linear(4500, 1)
        self.act3 = nn.Sigmoid()
    def forward(self, x):
        x = self.act1(self.conv1(x))
        #print("Conv1:", x.shape)
        x = self.pool1(x)
        #print("Pooling1:", x.shape)
        x = self.flat(x)
        #print("Flattened:", x.shape)
        x = self.fc1(x)
        x = self.act1(x)
        #print("FC1:", x.shape)
        x = self.fc2(x)
        #x = self.act3(x)
        #print("Output:", x.shape)
        return x

Setting the device

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Initialize the network

In [6]:
model = CNN().to(device)

Hyper-Parameters

In [4]:
in_channels = 3
num_classes = 2
learning_rate = 3e-4 # karpathy's constant
batch_size = 4
num_epochs = 3

Load the Data

In [5]:
trans_train =  transforms.Compose([transforms.Resize(75),
                                       transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(75),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()])

trans_valid = transforms.Compose([transforms.Resize(75),
                                      transforms.CenterCrop(75),
                                      transforms.ToTensor()])

data_dir = '/content/drive/MyDrive/Kaggle Datasets/Cat Dog'

train_dataset = datasets.ImageFolder(data_dir + '/train',
                                  transform=trans_train)
test_dataset = datasets.ImageFolder(data_dir + '/test',
                                 transform=trans_valid)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


Loss and optimizer

In [7]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


Train Network

In [8]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        #print(scores.shape, targets.shape)
        loss = criterion(scores, targets.reshape(-1,1).float())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples




100%|██████████| 138/138 [09:12<00:00,  4.01s/it]


In [9]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 50.27
Accuracy on test set: 48.12
